In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
datos_combi  = pd.read_csv(path_archivo)

In [3]:
datos_combi.tail()

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
52495,1.567655,1.570796,2.480205,0.003142,0.999995,0.043284,1.301064,1.417775,1.980252,9.517975,...,1522.778629,1545.897429,1549.270941,1555.514898,1592.310045,1607.108186,1638.655787,1647.559970,1664.940381,1681.654012
52496,1.567655,1.570796,2.645552,0.003142,0.999995,0.047054,1.226930,1.295361,1.877222,10.137011,...,1404.324881,1412.250358,1439.575747,1474.006907,1477.994348,1491.848290,1497.992037,1512.253541,1526.291159,1530.890602
52497,1.567655,1.570796,2.810899,0.003142,0.999995,0.050498,1.163447,1.186646,1.786764,10.574866,...,1318.615996,1335.348474,1346.952214,1363.197038,1391.893356,1394.572221,1413.658731,1421.467987,1425.394153,1429.766768
52498,1.567655,1.570796,2.976246,0.003142,0.999995,0.053558,1.088887,1.108445,1.706127,10.716328,...,1263.021825,1267.508608,1288.530625,1306.979513,1311.378390,1313.628527,1322.156960,1331.003658,1331.520897,1366.308756
52499,1.567655,1.570796,3.141593,0.003142,0.999995,0.056182,1.000000,1.060339,1.633308,10.473805,...,1191.013334,1211.355099,1229.992856,1255.484765,1255.901983,1258.454726,1258.454726,1300.430031,1300.430031,1304.301385


In [4]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [5]:
datos_combi.head()

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.002000,0.19635,1.047198,0.999998,0.002000,3.910766e-07,8.072690,13.084104,27.374015,33.406144,...,427443.935227,446916.371010,448827.125228,449977.613611,458726.245203,458940.189236,521446.826866,525588.873740,583373.344465,583607.080825
1,0.005138,0.19635,1.047198,0.999987,0.005138,1.003534e-06,8.072280,13.084550,27.402665,33.401889,...,427718.567842,447296.943246,449205.482004,450245.199365,459202.763203,459406.655524,521990.068226,526127.246576,583326.887609,583563.229874
2,0.008275,0.19635,1.047198,0.999966,0.008275,1.614694e-06,8.071871,13.084995,27.431316,33.397652,...,427993.234411,447677.888893,449584.234077,450511.485953,459679.284087,459873.134636,522533.310087,526665.556822,583279.715703,583518.646462
3,0.011413,0.19635,1.047198,0.999935,0.011413,2.224556e-06,8.071464,13.085440,27.459968,33.393435,...,428267.942931,448059.213137,449963.386266,450776.472028,460155.819486,460339.637726,523076.564859,527203.814226,583231.833940,583473.335731
4,0.014550,0.19635,1.047198,0.999894,0.014550,2.833116e-06,8.071059,13.085883,27.488622,33.389236,...,428542.698070,448440.917760,450342.939976,451040.152614,460632.377451,460806.172377,523619.840868,527742.024324,583183.242886,583427.298198


In [6]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [7]:
def graficar_lambdas(data_frame, target, n_eig):
    cols_eig = list(map(lambda x: "eig_" + str(x+1), range(n_eig)))
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = np.log(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    ax.set_xlabel(target)
    ax.set_ylabel("lambda moño")
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig)
    plt.savefig("pictures/eta=" +  eta + ", beta=" + beta + ".png")
    plt.close()
#fin for 

In [8]:
"""
for n in range(N_partes):
    start = N_phi_K*n
    end = N_phi_K*(n+1)
    graficar_lambdas(datos_combi.iloc[start:end], "phi_K", 3)
#fin for 
"""

'\nfor n in range(N_partes):\n    start = N_phi_K*n\n    end = N_phi_K*(n+1)\n    graficar_lambdas(datos_combi.iloc[start:end], "phi_K", 3)\n#fin for \n'

In [53]:
#Vamos a entrenar el modelo con TODOS los datos combinatoriales. Nuestros datos de test serán aleatorios. 
target = "phi_K"
N_eig = 2
features_eig = ["eig_0"] + list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
features_dim = ["eta", "beta"]
features = features_dim + features_eig
X_train = datos_combi[features]
y_train = datos_combi[target]

In [54]:
X_train.head()

,eta,beta,eig_0,eig_1,eig_2
0,0.19635,1.047198,3.910766e-07,8.072690,13.084104
1,0.19635,1.047198,1.003534e-06,8.072280,13.084550
2,0.19635,1.047198,1.614694e-06,8.071871,13.084995
3,0.19635,1.047198,2.224556e-06,8.071464,13.085440
4,0.19635,1.047198,2.833116e-06,8.071059,13.085883


In [56]:
def get_metrics(X, y, model):
    y_gorro = model.predict(X)
    R2 = r2_score(y, y_gorro)
    RMSE = root_mean_squared_error(y, y_gorro)
    MAE = mean_absolute_error(y, y_gorro)
    return {"R2": R2, "RMSE": RMSE, "MAE": MAE}
#fin función

class TransformadorDeDatos(BaseEstimator, TransformerMixin):
    def __init__(self, cols_tr, deg = 3):
        self.deg = deg
        self.cols_tr = cols_tr
    #fin init

    def fit(self, X, y=None):
        return self
    #fin fit

    def transform(self, X):
        datos = X.copy()
        cols_transformar = self.cols_tr
        poly = PolynomialFeatures(degree=self.deg, include_bias=False)
        poly_data = poly.fit_transform(datos[cols_transformar])
        poly_feature_names = poly.get_feature_names_out(input_features=cols_transformar)
        datos_finales = pd.DataFrame(poly_data, columns=poly_feature_names)
        return datos_finales
    #fin transform
#fin class

In [57]:
def crear_pipeline(modelo, grado, nombre_modelo, cols_tr):
    pipeline = Pipeline([('custom-transformer', TransformadorDeDatos(cols_tr, deg = grado)),
            ('power-transformer', PowerTransformer(method = "box-cox", standardize = True)), 
            (nombre_modelo, modelo)])
    return pipeline
#fin función

In [58]:
pipeline = crear_pipeline(LinearRegression(), 3, "re_lineal_simple", ["eig_0", "eig_1", "eig_2", "eta", "beta"])

In [59]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('custom-transformer',
                 TransformadorDeDatos(cols_tr=['eig_0', 'eig_1', 'eig_2', 'eta',
                                               'beta'])),
                ('power-transformer', PowerTransformer(method='box-cox')),
                ('re_lineal_simple', LinearRegression())])

In [60]:
metrs = get_metrics(X_train, y_train, pipeline)

In [61]:
print(metrs)

{'R2': 0.6735684916309566, 'RMSE': 0.2587438276890856, 'MAE': 0.18943096186150954}
